In [ ]:
# preprocessing.py

from transformers import BertTokenizer, TFBertModel
from joblib import load
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import tensorflow as tf

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

nltk.download('punkt')
nltk.download('stopwords')

def bert_feature_extraction(text):
    with tf.device('/GPU:0'):
        inputs = tokenizer.encode_plus(text, return_tensors='tf', add_special_tokens=True, max_length=50, truncation=True, padding='max_length')
        outputs = model(inputs)
        return outputs.last_hidden_state[:,0,:].numpy()

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)

    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]
    
    return ' '.join(tokens)


In [2]:
classifier = load('KOJA-CR.joblib')
def classify_sentence(sentence):
    # Preprocess the sentence
    preprocessed_sentence = preprocess_text(sentence)
    
    # Extract features using BERT
    features = bert_feature_extraction(preprocessed_sentence)
    
    # Reshape the features to match the input shape
    features = features.reshape(1, -1)
    
    # Make the prediction
    return classifier.predict(features)[0]

In [3]:
result = classify_sentence("Meeting with the marketing team")
print(result)

result = classify_sentence("Annual company picnic")
print(result)

result = classify_sentence("Project deadline")
print(result)

result = classify_sentence("Client presentation")
print(result)

result = classify_sentence("Team building workshop")
print(result)

result = classify_sentence("Quarterly business review")
print(result)

result = classify_sentence("Product launch event")
print(result)

result = classify_sentence("Board of directors meeting")
print(result)

result = classify_sentence("Employee training session")
print(result)

result = classify_sentence("Year-end party")
print(result)


Educational Events
Religious & Cultural Events
Other
Celebrations & Parties
Educational Events
Educational Events
Launch & Networking Events
Celebrations & Parties
Celebrations & Parties
Launch & Networking Events
